# Описание проекта
Коронавирус застал мир врасплох, изменив привычный порядок вещей. На какое-то время жители городов перестали выходить на улицу, посещать кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

In [1]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa
# устанавливаем параметры
db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

In [2]:
# формируем запрос и выводим данные
query = '''
SELECT * 
FROM books 
LIMIT 5
'''
get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
query = '''
SELECT * 
FROM authors 
LIMIT 5
'''
get_sql_data(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
query = '''
SELECT * 
FROM publishers 
LIMIT 5
'''
get_sql_data(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
query = '''
SELECT * 
FROM ratings 
LIMIT 5
'''
get_sql_data(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
query = '''
SELECT *
FROM reviews
LIMIT 5
'''
get_sql_data(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [7]:
query = '''
SELECT COUNT(*)
FROM reviews
'''
get_sql_data(query)

,count
0,2793


## Посчитайте, сколько книг вышло после 1 января 2000 года

In [8]:
query = '''
SELECT COUNT(*)
FROM books b
WHERE b.publication_date > '2000-01-01' 
'''
get_sql_data(query)

,count
0,819


## Для каждой книги посчитайте количество обзоров и среднюю оценку

In [20]:
query = '''
SELECT b.book_id,
       b.title,
       COUNT(DISTINCT(r2.review_id)) AS review_count,
       AVG(r.rating) AS average_rating
FROM books b 
LEFT JOIN ratings r on b.book_id = r.book_id
LEFT JOIN reviews r2 on r2.book_id = b.book_id 
GROUP BY b.book_id  
ORDER BY average_rating DESC
'''
get_sql_data(query)

,book_id,title,review_count,average_rating
0,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,901,The Walking Dead Book One (The Walking Dead #...,2,5.00
2,390,Light in August,2,5.00
3,972,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,136,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,316,His Excellency: George Washington,2,2.00
997,202,Drowning Ruth,3,2.00
998,371,Junky,2,2.00


## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [10]:
query = ''' 
SELECT publisher, 
       COUNT(book_id)
FROM publishers p 
JOIN books b on p.publisher_id =b.publisher_id 
WHERE num_pages>50 
GROUP BY publisher
ORDER BY COUNT(book_id) DESC 
LIMIT 1
'''
get_sql_data(query)

,publisher,count
0,Penguin Books,42


## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [11]:
query = '''
WITH br AS (
    SELECT 
        b.book_id,
        b.author_id,
        AVG(r.rating) AS average_rating,
        COUNT(r.rating_id) AS rating_count
    FROM 
        books b
    JOIN 
        ratings r ON b.book_id = r.book_id
    GROUP BY 
        b.book_id
    HAVING 
        COUNT(r.rating_id) >= 50
),
ar AS (
    SELECT 
        br.author_id,
        AVG(br.average_rating) AS author_average_rating
    FROM 
        br
    GROUP BY 
        br.author_id
)
SELECT 
    a.author, ar.author_average_rating
FROM 
    ar
JOIN 
    authors a ON ar.author_id = a.author_id
ORDER BY 
    ar.author_average_rating DESC
LIMIT 1
'''
get_sql_data(query)

,author,author_average_rating
0,J.K. Rowling/Mary GrandPré,4.283844


## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [12]:
query = '''
WITH urt AS (
    SELECT 
        username,
        COUNT(rating_id) AS rating_count
    FROM 
        ratings
    GROUP BY 
        username
    HAVING 
        COUNT(rating_id) > 48
),
ur AS (
    SELECT 
        u.username,
        COUNT(rv.review_id) AS review_count
    FROM 
        urt u
    JOIN 
        reviews rv ON u.username = rv.username
    GROUP BY 
        u.username
)
SELECT 
    AVG(review_count) AS average_reviews
FROM 
    ur
'''
get_sql_data(query)

,average_reviews
0,24.0


## Вывод

С 1 января 2000 года было выпущено 819 книг.  
Наименьшую оценку получила книга Harvesting the Heart  
Большее количество обзоров было написано для книги Twilight (Twilight #1) 	
Издательство, которое опубликовала наибольшее количество книг  - Penguin Books  
Среди самых популярных книг (с количеством оценок более 50) наивысшую оценку получили книги автора J.K. Rowling (средняя оценка - 4.3)  
В среднем пользователи, поставившие более 48 оценок книгам, также написали 24 обзора.
